# Hotel Booking Cancellation Prediction using KNN

## Objective:
To predict whether a hotel booking will be cancelled or not using the K-Nearest Neighbors (KNN) classification algorithm.


In [ ]:
!pip install snowflake-connector-python

In [ ]:
import pandas as pd
import snowflake.connector

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
conn = snowflake.connector.connect(
    user = 'SANTHIYA347',
    password = 'Santhiya@123456',
    account = 'RTFFDMY-GG82011',
    database ='HOTEL',
    schema ='PUBLIC',
    warehouse = 'COMPUTE_WH'
)

In [ ]:
#run query
query = 'SELECT * FROM "HOTEL_BOOK"'
df=pd.read_sql(query, conn)

#close conn
conn.close()

#preview
print(df.head())

/tmp/ipython-input-3893334273.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql(query, conn)


          HOTEL  IS_CANCELED  LEAD_TIME  ARRIVAL_DATE_YEAR ARRIVAL_DATE_MONTH  \
0  Resort Hotel            0        342               2015               July   
1  Resort Hotel            0        737               2015               July   
2  Resort Hotel            0          7               2015               July   
3  Resort Hotel            0         13               2015               July   
4  Resort Hotel            0         14               2015               July   

   ARRIVAL_DATE_WEEK_NUMBER  ARRIVAL_DATE_DAY_OF_MONTH  \
0                        27                          1   
1                        27                          1   
2                        27                          1   
3                        27                          1   
4                        27                          1   

   STAYS_IN_WEEKEND_NIGHTS  STAYS_IN_WEEK_NIGHTS  ADULTS  ...  CUSTOMER_TYPE  \
0                        0                     0       2  ...      Transient   
1   

In [ ]:
drop_cols = [
    'name', 'email', 'phone-number', 'credit_card',
    'reservation_status', 'reservation_status_date'
]

df.drop(columns=drop_cols, inplace=True, errors='ignore')

In [ ]:
print(df.shape)
print(df.columns)
print(df.describe())
print(df.info())

(119390, 35)
Index(['HOTEL', 'IS_CANCELED', 'LEAD_TIME', 'ARRIVAL_DATE_YEAR',
       'ARRIVAL_DATE_MONTH', 'ARRIVAL_DATE_WEEK_NUMBER',
       'ARRIVAL_DATE_DAY_OF_MONTH', 'STAYS_IN_WEEKEND_NIGHTS',
       'STAYS_IN_WEEK_NIGHTS', 'ADULTS', 'CHILDREN', 'BABIES', 'MEAL',
       'COUNTRY', 'MARKET_SEGMENT', 'DISTRIBUTION_CHANNEL',
       'IS_REPEATED_GUEST', 'PREVIOUS_CANCELLATIONS',
       'PREVIOUS_BOOKINGS_NOT_CANCELED', 'RESERVED_ROOM_TYPE',
       'ASSIGNED_ROOM_TYPE', 'BOOKING_CHANGES', 'DEPOSIT_TYPE', 'AGENT',
       'COMPANY', 'DAYS_IN_WAITING_LIST', 'CUSTOMER_TYPE', 'ADR',
       'REQUIRED_CAR_PARKING_SPACES', 'TOTAL_OF_SPECIAL_REQUESTS',
       'RESERVATION_STATUS', 'RESERVATION_STATUS_DATE', 'NAME', 'EMAIL',
       'CREDIT_CARD'],
      dtype='object')
         IS_CANCELED      LEAD_TIME  ARRIVAL_DATE_YEAR  \
count  119390.000000  119390.000000      119390.000000   
mean        0.370416     104.011416        2016.156554   
std         0.482918     106.863097           0.707476  

In [ ]:
df.isnull().sum()

,0
HOTEL,0
IS_CANCELED,0
LEAD_TIME,0
ARRIVAL_DATE_YEAR,0
ARRIVAL_DATE_MONTH,0
ARRIVAL_DATE_WEEK_NUMBER,0
ARRIVAL_DATE_DAY_OF_MONTH,0
STAYS_IN_WEEKEND_NIGHTS,0
STAYS_IN_WEEK_NIGHTS,0
ADULTS,0


In [ ]:
# Numerical → median
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

# Categorical → mode
cat_cols = df.select_dtypes(include=['object']).columns
df[cat_cols] = df[cat_cols].fillna(df[cat_cols].mode().iloc[0])


In [ ]:

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for col in cat_cols:
    df[col] = le.fit_transform(df[col])


In [ ]:
df.head()

,HOTEL,IS_CANCELED,LEAD_TIME,ARRIVAL_DATE_YEAR,ARRIVAL_DATE_MONTH,ARRIVAL_DATE_WEEK_NUMBER,ARRIVAL_DATE_DAY_OF_MONTH,STAYS_IN_WEEKEND_NIGHTS,STAYS_IN_WEEK_NIGHTS,ADULTS,...,DAYS_IN_WAITING_LIST,CUSTOMER_TYPE,ADR,REQUIRED_CAR_PARKING_SPACES,TOTAL_OF_SPECIAL_REQUESTS,RESERVATION_STATUS,RESERVATION_STATUS_DATE,NAME,EMAIL,CREDIT_CARD
0,1,0,342,2015,5,27,1,0,0,2,...,0,2,0.0,0,0,1,121,27015,35035,3322
1,1,0,737,2015,5,27,1,0,0,2,...,0,2,0.0,0,0,1,121,3977,5471,8157
2,1,0,7,2015,5,27,1,0,1,1,...,0,2,75.0,0,0,1,122,64151,90298,2734
3,1,0,13,2015,5,27,1,0,1,1,...,0,2,75.0,0,0,1,122,47596,65652,4677
4,1,0,14,2015,5,27,1,0,2,2,...,0,2,98.0,0,1,1,123,48754,64565,4498


In [ ]:
X=df.drop(['IS_CANCELED'],axis=1)
y=df['IS_CANCELED']

In [ ]:
print(X.head())
print(y.head())

   HOTEL  LEAD_TIME  ARRIVAL_DATE_YEAR  ARRIVAL_DATE_MONTH  \
0      1        342               2015                   5   
1      1        737               2015                   5   
2      1          7               2015                   5   
3      1         13               2015                   5   
4      1         14               2015                   5   

   ARRIVAL_DATE_WEEK_NUMBER  ARRIVAL_DATE_DAY_OF_MONTH  \
0                        27                          1   
1                        27                          1   
2                        27                          1   
3                        27                          1   
4                        27                          1   

   STAYS_IN_WEEKEND_NIGHTS  STAYS_IN_WEEK_NIGHTS  ADULTS  CHILDREN  ...  \
0                        0                     0       2       0.0  ...   
1                        0                     0       2       0.0  ...   
2                        0                     1     

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(95512, 34)
(23878, 34)
(95512,)
(23878,)


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(
    n_neighbors=7,
    metric='euclidean'
)

knn.fit(X_train, y_train)


KNeighborsClassifier(metric='euclidean', n_neighbors=7)

In [ ]:
y_pred = knn.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.9830387804673758

Confusion Matrix:
 [[14874    33]
 [  372  8599]]

Classification Report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99     14907
           1       1.00      0.96      0.98      8971

    accuracy                           0.98     23878
   macro avg       0.99      0.98      0.98     23878
weighted avg       0.98      0.98      0.98     23878



## Conclusion:
The KNN model was able to predict hotel booking cancellations with reasonable accuracy.
Feature scaling significantly improved model performance, highlighting its importance
in distance-based algorithms like KNN.
